#### 导入库

In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

#### 文档加载&切分&检索

In [2]:
! python ingest.py # 命令行 python3 ./DB/Process/ingest.py 制作向量数据库

python: can't open file '/home/linjw/AGI/llama2/langchain/ingest.py': [Errno 2] No such file or directory


In [3]:
### 文档查询 去向量数据库匹配相关代码
doc_query = "未来 将要 细分赛道 分化 五年 产业规模 子领域"


### 递归总结, 根据查询出来的向量数据库相关片段 循环输入进llm进行要素提取
refine_query = """
根据获取的信息,总结并推理提炼出人工智能5年后的细分赛道的产业规模:
1. 技术1和其产业规模: [描述]
2. 技术2和其产业规模: [描述]
3. 技术3和其产业规模: [描述]
4. 技术4和其产业规模: [描述]
5. 技术5和其产业规模: [描述]
"""


# 最终总结输出 总结递归的最后输出
summary_query = """在已知的信息中，确定并列出人工智能5年后的细分赛道的产业规模"""


# 回答模板 将summary_query的输出格式化为对象
response_schemas = [
 ResponseSchema(name="技术1/产业规模", description="技术1和其产业规模"),
 ResponseSchema(name="技术2/产业规模", description="技术2和其产业规模"),
 ResponseSchema(name="技术3/产业规模", description="技术3和其产业规模"),
 ResponseSchema(name="技术4/产业规模", description="技术4和其产业规模"),
 ResponseSchema(name="技术5/产业规模", description="技术5和其产业规模"),]



In [4]:
sys.path.append(os.path.dirname(os.path.realpath('__file__'))+"/../")
import config.conf as cfg
from chromadb.config import Settings

# Define the folder for storing database
PERSIST_DIRECTORY = cfg.PERSIST_DIRECTORY.replace("/langchain","")
# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)

embeddings_model_name = cfg.EMBEDDINGS_MODEL_NAME.replace("/langchain","")
persist_directory = cfg.PERSIST_DIRECTORY.replace("/langchain","")
model_path = cfg.MODEL_PATH.replace("/langchain","")
model_n_ctx = cfg.MODEL_N_CTX
model_n_batch = cfg.MODEL_N_BATCH
target_source_chunks = 4
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)

In [5]:
cfg.EMBEDDINGS_MODEL_NAME

'/home/linjw/AGI/llama2/langchain/checkpoints/paraphrase-multilingual-MiniLM-L12-v2'

##### 普通检索

In [6]:
retriever = db.as_retriever(search_kwargs={"k": 30})

# retriever = db.as_retriever(    
#     search_type="mmr",
#     search_kwargs={'k': 20, 'lambda_mult': 0.25})


In [7]:
docs = retriever.get_relevant_documents(doc_query)
[doc.page_content for doc in docs]

['37 \n \n报告编号[19RI0028] \n未来，伴随着技术迭代与生态格局变化，拥有雄厚技术架构实力与解决方案落地能力的\n企业将蚕食部分中小企业份额，技术薄弱的中小公司将进一步被淘汰出局，行业将进入整合\n周期，行业格局持续优化，预计未来智慧交通市场竞争格局将会更加激烈。 \n6.2 \n银江股份有限公司 \n6.2.1 企业概况 \n银江股份有限公司（以下简称银江）成立于 1992 年，是中国领先的城市智能化整体解\n决方案运营商，于 2009 年在上海证券交易所上市，正式登陆 A 股。银江以智能化系统工程\n产品为主营业务，包括城市交通智能化、医疗信息化和建筑智能化三类。在城市交通智能化\n领域，银江通过城市交通智能化管控综合配套系统、城市交通智能化诱导系统为客户提供智\n慧交通一体化的解决方案。经过二十多年发展，银江建立了国家级企业技术中心、国家级博\n士后科研工作站、院士专家工作站，浙江省智能交通工程技术研究中心等技术创新平台。 \n凭借研发创新与资金实力优势，银江在城市智能化解决方案方面拥有专利总量约 300',
 '5G+云+AI三者相互协同，将打破众多行业发展天花板，打开新的成长空间，同时释放出巨大能\n量，为政企转型和产业升级注入新活力。\n融合视角：5G+云计算+AI深度融合协作创新，构筑数字经济\n动力引擎\n38%\n43%\n49%\n55%\n61%\n0%\n10%\n20%\n30%\n40%\n50%\n60%\n70%\n2019\n2020E\n2021E\n2022E\n2023E\n来源：国务院发展研究中心，亿欧智库整理\n亿欧智库：2019-2023年中国政府和大型企业\n上云率\n[INTEGRATION\n]',
 '映在业务层面可以重点关注客户拓展与模式创新带来的收入增长，\n以及规模效应逐步显现推动费用率显著下降； \n\uf0d8 路径的选择：人工智能场景落地复杂且业务链条长，我们判断上述\n厂商势必需要进行一定的取舍，具体有两条路径：一是沿着场景进\n行取舍，从而产生业务场景的垂直化；二是沿着产业链环节进行取\n舍，从而带来产业链的分工化。 \n■风险提示：行业增长不及预期、行业竞争格局恶化。 \nTable_Tit le \n2022 年 01 月 14 日 \n传媒 \nTable_BaseI n

In [8]:
for doc in docs:
    print(doc.metadata)


{'author': '', 'creationDate': '', 'creator': '', 'file_path': '/home/linjw/AGI/llama2/DB/Process/source/2019年  智慧交通研究——人工智能技术助力行业智能化升级，前景可期-头豹研究院.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 36, 'producer': '', 'source': '/home/linjw/AGI/llama2/DB/Process/source/2019年  智慧交通研究——人工智能技术助力行业智能化升级，前景可期-头豹研究院.pdf', 'subject': '', 'title': '', 'total_pages': 44, 'trapped': ''}
{'author': '张一豪', 'creationDate': "D:20200709192943+08'00'", 'creator': 'Microsoft® PowerPoint® 适用于 Microsoft 365', 'file_path': '/home/linjw/AGI/llama2/DB/Process/source/2020年  亿欧智库：中国人工智能商业落地研究报告.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20200709192943+08'00'", 'page': 8, 'producer': 'Microsoft® PowerPoint® 适用于 Microsoft 365', 'source': '/home/linjw/AGI/llama2/DB/Process/source/2020年  亿欧智库：中国人工智能商业落地研究报告.pdf', 'subject': '', 'title': '2020中国人工智能商业落地研究报告', 'total_pages': 79, 'trapped': ''}
{'author': '', 'creationDate': '', 'creator': '', 'file_path': '/home/linjw/AGI/llama2/DB/Proc

##### LLM压缩检索

In [9]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain.llms import OpenAI
# from langchain.retrievers.self_query.base import SelfQueryRetriever
# from langchain.chains.query_constructor.base import AttributeInfo
 
# #定义元数据的过滤条件
# metadata_field_info = [
#     AttributeInfo(
#         name="source",
#         description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
#         type="string",
#     ),
#     AttributeInfo(
#         name="page",
#         description="The page from the lecture",
#         type="integer",
#     ),
# ]
 
# #创建SelfQueryRetriever
# document_content_description = "Lecture notes"
# llm = OpenAI(temperature=0)
# retriever = SelfQueryRetriever.from_llm(
#     llm,
#     db,
#     document_content_description,
#     metadata_field_info,
#     verbose=True
# )
# def pretty_print_docs(docs):
#     print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))
    
# # Wrap our vectorstore
# llm = OpenAI(temperature=0)
# compressor = LLMChainExtractor.from_llm(llm)
 
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor,
#     base_retriever=db.as_retriever()
# )

# question = "what did they say about matlab?"
# compressed_docs = compression_retriever.get_relevant_documents(question)
# pretty_print_docs(compressed_docs)

#### 加载模型

In [10]:
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
model = HuggingFacePipeline.from_model_id(model_id=model_path,
        task="text-generation",
        device=0,
        pipeline_kwargs={
            "max_new_tokens": 4096, 
            "do_sample": True,
            "temperature": 0.2,
            "top_k": 1,
            "top_p": 0.9,
            "repetition_penalty": 1.1},
        model_kwargs={
            "torch_dtype": "auto",
            "low_cpu_mem_usage": True}
        )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


##### 定义目标模板

In [11]:
qa_chain_refine = RetrievalQA.from_chain_type(
    model,
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type="refine"
)
 
result = qa_chain_refine({"query": refine_query})
result["result"]


'\n\nRevised Answer:\n\n1. 技术1和其产业规模: [描述]\n2. 技术2和其产业规模: [描述]\n3. 技术3和其产业规模: [描述]\n4. 技术4和其产业规模: [描述]\n5. 技术5和其产业规模: [描述]'

### 整理回答

#### 数据解析器

In [12]:
que =result["result"]

initial_qa_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "以下为背景知识：\n"
    "{context_str}"
    "\n"
    "{format_instructions}"
    "\n"
    "请根据以上背景知识, 回答这个问题：{question}。"
    " [/INST]"
)


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template=initial_qa_template,
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions,"context_str":que}
)
_input = prompt.format_prompt(question=summary_query)
output = model(_input.to_string())
print(output)
output_parser.parse(output)
 

/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 以下是人工智能5年后可能存在的一些细分赛道及其对应的产业规模预测：

1. 自动驾驶汽车（Autonomous Vehicles）-预计到2025年，全球自动驾驶汽车市场规模将达到约1.5万亿美元。
2. 智能医疗保健（Smart Healthcare）-预计到2025年，全球智能医疗保健市场规模将超过1.6万亿美元。
3. 智能家居与物联网（Smart Home & IoT）-预计到2025年，全球智能家居与物联网市场规模将超过1.8万亿美元。
4. 机器人与自动化（Robotics & Automation）-预计到2025年，全球机器人与自动化市场规模将超过2.5万亿美元。
5. 虚拟现实与增强现实（Virtual Reality & Augmented Reality）-预计到2025年，全球虚拟现实与增强现实市场规模将超过1.2万亿美元。

请注意，这些数据仅供参考，实际结果可能会受到多种因素的影响而有所变化。


OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)